In [1]:
from Crypto.Util.number import inverse,GCD,isPrime,getPrime
from gmpy2 import gcdext,iroot,isqrt
from random import randint

In [2]:
# get n,e,c in all frames
dct = {}
for i in range(21):
    f = open("./frame/Frame{}".format(i),'r')
    s = f.read()
    temp = [int(s[i*256:(i+1)*256],16) for i in range(3)]
    dct["frame{}".format(i)] = temp

In [3]:
# 能分解n的都调用这个解密
def decrypt(p,q,n,e,c):
    phi = (p-1)*(q-1)
    d = inverse(e,phi)
    m = pow(c,d,n)
    print(bytes.fromhex(hex(m)[2:]))

## common modulus

In [4]:
# find frames that could be attacked
for i in range(len(dct)-1):
    for j in range(i+1,len(dct)):
        f1 = dct["frame{}".format(i)]
        f2 = dct["frame{}".format(j)]
        if f1[0] == f2[0]:
            if GCD(f1[1],f2[1]) == 1:
                print(i,j)

0 4


In [5]:
def common_modulus(e1, e2, n, c1, c2):
    # 调用扩展的欧几里得算法函数
    g, s, t = gcdext(e1, e2)
    # 求模反元素
    if s < 0:
        s = -s
        c1 = inverse(c1,n)
    if t < 0:
        t = -t
        c2 = inverse(c2,n)
    return pow(c1,s,n)*pow(c2,t,n)%n

In [6]:
f1 = dct["frame{}".format(0)]
f2 = dct["frame{}".format(4)]
m = common_modulus(f1[1],f2[1],f1[0],f1[2],f2[2])
bytes.fromhex(hex(m)[2:])

b'\x98vT2\x10\xab\xcd\xef\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00My secre'

## 模不互素

In [7]:
# find frames that could be attacked
for i in range(len(dct)-1):
    for j in range(i+1,len(dct)):
        f1 = dct["frame{}".format(i)]
        f2 = dct["frame{}".format(j)]
        p = GCD(f1[0],f2[0])
        if f1[0] != f2[0] and p != 1:
            print(i,j)
            f1 = dct["frame{}".format(i)]
            f2 = dct["frame{}".format(j)]
            decrypt(p,f1[0]//p,f1[0],f1[1],f1[2])
            decrypt(p,f2[0]//p,f2[0],f2[1],f2[2])

1 18
b'\x98vT2\x10\xab\xcd\xef\x00\x00\x00\x0b\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00. Imagin'
b'\x98vT2\x10\xab\xcd\xef\x00\x00\x00\n\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00m A to B'


## Low Public Exponent
### brute_force_crack

In [8]:
# find frames that could be attacked
lst_low_e = []
for i in range(len(dct)):
    e = dct["frame{}".format(i)][1]
    if e<10:
        print(i,e)
        lst_low_e.append(i)

3 5
7 3
8 5
11 3
12 5
15 3
16 5
20 5


In [9]:
def brute_force_crack(n,e,c,repeat = 1000):
    m = 0
    for i in range(repeat):
        m, flag = iroot(c+i*n,e)
        if flag and pow(m,e,n) == c:
            return int(m) 
    return 0

In [10]:
for i in lst_low_e:
    f = dct["frame{}".format(i)]
    m = brute_force_crack(f[0],f[1],f[2])
    if m!=0: 
        print(i,int.to_bytes(m,48,'big'))

## Factoring Large Integers  
### Fermat's factorization method

In [11]:
def Fermat_factorization(num,repeat = 100):
    x,flag_x = iroot(num,2)
    if pow(x,2)<num:
        x+=1
    for i in range(repeat):
        temp = pow(x,2) - num
        y,flag_y = iroot(temp,2)
        if flag_y:
            return int((x+y)),int((x-y))  
        x+=1
    else:
        return 0,0

In [12]:
for i in range(21):
    f = dct["frame{}".format(i)]
    n = f[0]

    p,q = Fermat_factorization(n)
    if p and q:
        print(i)
        decrypt(p,q,n,e=f[1],c=f[2])

10
b'\x98vT2\x10\xab\xcd\xef\x00\x00\x00\x08\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00will get'


## Factoring Large Integers  
### Pollard's p-1 algorithm

In [13]:
def pollard_p_1(n,B=120000):
    a = 2
    false_range = int(0.8*B)
    for j in range(2,false_range):
        a = pow(a, j, n)

    d = 0
    for j in range(false_range,B+1):
        a = pow(a, j, n)
        d = GCD(a-1, n)
        if 1<d<n:
            return d

In [14]:
"""这里的数据太大了,要跑好久,就直接将结果放在下一个代码块中,不跑了"""
# lst_index_p = []
# for i in [2,6,10,14,19]:
#     f = dct["frame{}".format(i)]
#     n = f[0]
#     p = pollard_p_1(n)
#     if p:
#         lst_index_p.append((i,p))

# lst_index_p

'这里的数据太大了,要跑好久,就直接将结果放在下一个代码块中,不跑了'

In [15]:
lst_index_p = [(2,1719620105458406433483340568317543019584575635895742560438771105058321655238562613083979651479555788009994557822024565226932906295208262756822275663694111),(19, 1085663496559),(6, 920724637201)]

In [16]:
for i in lst_index_p:
    print(i[0])
    f =  dct["frame{}".format(i[0])]
    n = f[0]
    p = i[1]
    q = n//p
    decrypt(p,q,n,e = f[1],c = f[2])

2
b'\x98vT2\x10\xab\xcd\xef\x00\x00\x00\x06\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00 That is'
19
b'\x98vT2\x10\xab\xcd\xef\x00\x00\x00\x05\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00instein.'
6
b'\x98vT2\x10\xab\xcd\xef\x00\x00\x00\x07\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00 "Logic '


## Factoring Large Integers  
### Pollard rho

In [17]:
def pollard_rho(n):
    while True:
        r = randint(2,7)
        x = randint(2,n-1)
        y = pow(x,2,n)+r

        fac = GCD(abs(y-x),n)

        if fac>1 and isPrime(fac):
            return fac

# p = getPrime(10)
# q = getPrime(10)
# print(p,q,pollard_rho(p*q))

## Factoring Large Integers  
### Williams's p + 1 algorithm

In [18]:
# 生成lucas序列
def Lucas(A, N, pos):
    v0 = 2
    v1 = A
    if pos == 0:
        return v0
    elif pos == 1:
        return v1
    else:
        for _ in range(pos-1):
            v0,v1 = v1,(A*v1-v0)%N
        return v1

def williams_p_1(n,a,repeat = 20):
    i_fac = 1 # 阶乘
    for i in range(1,repeat):
        i_fac *= i
        lucas = Lucas(a,n,i_fac)%n
        p = GCD(lucas-2,n)
        if p>1 and p<n: 
            return p
            
# print(williams_p_1(112729,9))
# print(williams_p_1(112729,5))

In [19]:
# 展开为连分数
def continuedFra(x, y):
    cF = []
    while y:
        cF += [x // y]
        x, y = y, x % y
    return cF

def Simplify(ctnf):
    numerator = 0
    denominator = 1
    for x in ctnf[::-1]:
        numerator, denominator = denominator, x * denominator + numerator
    return (numerator, denominator)

# 连分数化简
def calculateFrac(x, y):
    cF = continuedFra(x, y)
    cF = list(map(Simplify, (cF[0:i] for i in range(1, len(cF)))))
    return cF

# 解韦达定理
def solve_pq(a, b, c):
    par = isqrt(abs(b * b - 4 * a * c))
    return (-b + par) // (2 * a), (-b - par) // (2 * a)

def wienerAttack(e, n):
    for (d, k) in calculateFrac(e, n):
        if k == 0: continue
        if (e * d - 1) % k != 0: continue

        phi = (e * d - 1) // k
        p, q = solve_pq(1, n - phi + 1, n)
        if p * q == n:
            return abs(int(p)), abs(int(q))
    return 0,0

In [20]:
n = 12238605063252292170613110607692779326628090745751955692266649177882959231822580682548279800443278979485092243645806337103841086023159482786712759291169541633901936290854044069486201989034158882661270017305064348254800318759062921744741432214818915527537124001063995865927527037625277330117588414586505635959411443039463168463608235165929831344586283875119363703480280602514451713723663297066810128769907278246434745483846869482536367912810637275405943566734099622063142293421936734750356828712268385319217225803602442033960930413469179550331907541244416573641309943913383658451409219852933526106735587605884499707827
e = 11850552481503020257392808424743510851763548184936536180317707155841959788151862976445957810691568475609821000653594584717037528429828330763571556164988619635320288125983463358648887090031957900011546300841211712664477474767941406651977784177969001025954167441377912326806132232375497798238928464025466905201977180541053129691501120197010080001677260814313906843670652972019631997467352264392296894192998971542816081534808106792758008676039929763345402657578681818891775091140555977382868531202964486261123748663752490909455324860302967636149379567988941803701512680099398021640317868259975961261408500449965277690517
p,q = wienerAttack(e,n)
print("p: {}\nq: {}".format(p,q))
if p and q: 
    phi = (p-1)*(q-1)
    d = inverse(e,phi)
    print("d: {}".format(d))


p: 110628229052318704152246095359270943887813104444949254398128993373602031238362003253725157377156430190674485981957445623074380516333901046053625898418396428434438992000260593732179397554401650954591549716160461307807121354462523624398848957270672460052953336283536148203105186509336678669629895378279072978393
q: 110628229052318704152246095359270943887813104444949254398128993373602031238362003253725157377156430190674485981957445623074380516333901046053625898418396428434438992000260593732179397554401650954591549716160461307807121354462523624398848957270672460052953336283536148203105186509336678669629895378279072978539
d: 392321823339749506721829211885


## coppersmith 相关攻击
### broadcast attack

In [21]:
# find frames that could be attacked
for i in range(len(dct)):
    e = dct["frame{}".format(i)][1]
    if e<10:
        print(i,e)

3 5
7 3
8 5
11 3
12 5
15 3
16 5
20 5


In [22]:
# crt
def egcd(a, b):
    if a == 0:
        return (b, 0, 1)
    else:
        g, y, x = egcd(b % a, a)
        y, x = x - (b // a) * y, y
        return (g, y, x)

def is_prime(arr):
    for i in range(len(arr)-1):
        for j in range(i+1,len(arr)):
            if egcd(arr[i],arr[j])[0]!=1:
                return False
            return True

def mul(arr):
    end = 1
    for i in arr:
        end*=i
    return end

def crt(lst_a,lst_m):
    product_m=mul(lst_m)
    x = 0
    M = [product_m // i for i in lst_m]
    C = []
    for i in range(len(M)):
        C.append(egcd(M[i], lst_m[i])[1])
    
    for i in range(len(M)):
        x += lst_a[i] * M[i] * C[i]
    return x%product_m

# print(crt([2,3,2],[3,5,7]))

In [23]:
lst_common_e = [3,8,12,16,20]
# lst_common_e = [7,11,15]
lst_a = []
lst_m = []
for i in lst_common_e:
    f = dct["frame{}".format(i)]
    lst_a.append(f[2])
    lst_m.append(f[0])
    e = f[1]
m_e = crt(lst_a,lst_m)
m = iroot(m_e,e)[0]
print(bytes.fromhex(hex(m)[2:]))

b'\x98vT2\x10\xab\xcd\xef\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00t is a f'


## coppersmith 相关攻击
### 已知部分明文

- 以下代码在sage环境下使用
- 将e = 3的(n,c)分别替换下述代码中的(n,c)即可

```python
n = 155266493936043103849855199987896813716831986416707080645036022909153373110367007140301635144950634879983289720164117794783088845393686109145443728632527874768524615377182297125716276153800765906014206797548230661764274997562670900115383324605843933035314110752560290540848152237316752573471110899212429555149
c = 124929943232081828105808318993257526364596580021564021377503915670544445679836588765369503919311404328043203272693851622132258819278328852726005776082575583793735570095307898828254568015886630010269615546857335790791577865565021730890364239443651479580968112031521485174068731577348690810906553798608040451024
e = 3

# 枚举可能的tag
m_lst = []
s = "9876543210abcdef0000000b00000000000000000000000000000000000000000000000000000000000000000000000000000000000000002e20496d6167696e"
for i in "1234567890abcdef":
    m_lst.append(eval('0x'+s[:23]+i+s[24:]))

# 破解
for m in m_lst:
    # 获取m中已知部分,未知部分用0替代
    nbits = 512
    kbits = 64
    mbar = m&(pow(2,nbits)-pow(2,kbits))

    #Zmod(n):Zmod代表这是一个整数域中的n模环
    #PolynomialRing：这个就是说建立多项式环
    #.<X>：指定一个变量
    PR.<x> = PolynomialRing(Zmod(n))
    f = (mbar + x)**e - c
    # small_roots ( self , X = None , beta = 1.0 , epsilon = None , ** kwds )
    # X – 根的绝对边界
    roots = f.small_roots()
    if roots: 
        print(bytes.fromhex(hex(mbar+roots[0])[2:]))
        break
```

- 7
b'\x98vT2\x10\xab\xcd\xef\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00amous sa'

- 11
b'\x98vT2\x10\xab\xcd\xef\x00\x00\x00\x03\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00ying of '

- 15
b'\x98vT2\x10\xab\xcd\xef\x00\x00\x00\x04\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00Albert E'